### Basic working of Google Palm LLM in LangChain


In [2]:
from langchain.llms import GooglePalm
from dotenv import load_dotenv
import os
# get this free api key from https://makersuite.google.com/

load_dotenv()  # take environment variables from .env (especially openai api key)

llm = GooglePalm(google_api_key=os.environ["GOOGLE_API_KEY"], temperature=0.1)

In [3]:
poem = llm("Write a 4 line poem of my love for samosa")
print(poem)

/Users/meet/anaconda3/envs/omdena/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


**Oh, samosa, my love**
**You are so crispy and golden brown**
**With your filling of spiced potatoes**
**You are the perfect snack**


In [4]:
essay = llm("write email requesting refund for electronic item")
print(essay)

Dear [Name of Customer Service Representative],

I am writing to request a refund for the [product name] that I purchased on [date]. I am not satisfied with the product because [explain why you are not satisfied].

I have attached a copy of my receipt and a picture of the product. I have also included a link to the product's listing on your website.

I would like to request a full refund for the product. I would also like to request that you send me a replacement product.

I have been a loyal customer of your company for many years, and I have never had a problem with a product before. I am disappointed that I am not satisfied with this product, but I am confident that you will be able to resolve this issue to my satisfaction.

Thank you for your time and attention to this matter.

Sincerely,
[Your Name]


In [5]:
from langchain.chains import RetrievalQA


from langchain.embeddings import GooglePalmEmbeddings
from langchain.llms import GooglePalm

### Now let's load data from Codebasics FAQ csv file


In [6]:
from langchain.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(file_path='basic_faqs.csv', source_column="prompt")

# Store the loaded data in the 'data' variable
data = loader.load()

### Hugging Face Embeddings


In [7]:
from InstructorEmbedding import INSTRUCTOR
model = INSTRUCTOR('hkunlp/instructor-large')
sentence = "3D ActionSLAM: wearable person tracking in multi-floor environments"
instruction = "Represent the Science title:"
embeddings = model.encode([[instruction,sentence]])
print(embeddings)

load INSTRUCTOR_Transformer
max_seq_length  512
[[-6.15552627e-02  1.04199704e-02  5.88438474e-03  1.93768851e-02
   5.71417809e-02  2.57655438e-02 -4.01949983e-05 -2.80044544e-02
  -2.92965565e-02  4.91884872e-02  6.78200200e-02  2.18692329e-02
   4.54528667e-02  1.50187155e-02 -4.84451763e-02 -3.25259715e-02
  -3.56492773e-02  1.19935405e-02 -6.83917757e-03  3.03126313e-02
   5.17491512e-02  3.48140411e-02  4.91032843e-03  6.68928549e-02
   1.52824540e-02  3.54217142e-02  1.07743582e-02  6.89828768e-02
   4.44019474e-02 -3.23419608e-02  1.24268020e-02 -2.15528086e-02
  -1.62690766e-02 -4.15058173e-02 -2.42291158e-03 -3.07157822e-03
   4.27047275e-02  1.56428572e-02  2.57812925e-02  5.92843145e-02
  -1.99174173e-02  1.32361818e-02  1.08408015e-02 -4.00610566e-02
  -1.36213051e-03 -1.57032814e-02 -2.53812131e-02 -1.31972972e-02
  -7.83779565e-03 -1.14009101e-02 -4.82025519e-02 -2.58416049e-02
  -4.98769898e-03  4.98239547e-02  1.19490270e-02 -5.55060506e-02
  -2.82120295e-02 -3.3220872

In [8]:
from langchain_community.embeddings import HuggingFaceInstructEmbeddings

# Initialize instructor embeddings using the Hugging Face model
instructor_embeddings = HuggingFaceInstructEmbeddings(model_name='hkunlp/instructor-large')

e = instructor_embeddings.embed_query("What is your refund policy?") 

load INSTRUCTOR_Transformer
max_seq_length  512


In [9]:
len(e)

768

In [10]:
e[:5]

[-0.043898068368434906,
 0.007685536053031683,
 -0.00923190638422966,
 0.024496261030435562,
 0.033592283725738525]

As you can see above, embedding for a sentance "What is your refund policy" is a list of size 768. Looking at the numbers in this list, doesn't give any intuitive understanding of what it is but just assume that these numbers are capturing the meaning of "What is your refund policy". If you are curious to know about embeddings, go to youtube and search "codebasics word embeddings" and you will find bunch of videos with simple, intuitive explanations


### Vector store using FAISS


In [11]:
from langchain_community.vectorstores import FAISS

# Create a FAISS instance for vector database from 'data'
vectordb = FAISS.from_documents(documents=data,
                                 embedding=instructor_embeddings)

# Create a retriever for querying the vector database
retriever = vectordb.as_retriever(score_threshold = 0.7)

In [23]:
# Save the FAISS index to an external file
#vectordb.save_local("faiss_index")

In [12]:
rdocs = retriever.get_relevant_documents("how about job placement support?")
rdocs

[Document(page_content='prompt: Do you provide any job assistance?\nresponse: Yes, We help you with resume and interview preparation along with that we help you in building online credibility, and based on requirements we refer candidates to potential recruiters.', metadata={'source': 'Do you provide any job assistance?', 'row': 11}),
 Document(page_content='prompt: Will this course guarantee me a job?\nresponse: We created a much lighter version of this course on YouTube available for free (click this link) and many people gave us feedback that they were able to fetch jobs (see testimonials). Now this paid course is at least 5x better than the YouTube course which gives us ample confidence that you will be able to get a job. However, we want to be honest and do not want to make any impractical promises! Our guarantee is to prepare you for the job market by teaching the most relevant skills, knowledge & timeless principles good enough to fetch the job.', metadata={'source': 'Will this 

As you can see above, the retriever that was created using FAISS and hugging face embedding is now capable of pulling relavant documents from our original CSV file knowledge store. This is very powerful and it will help us further in our project


##### Embeddings can be created using GooglePalm too. Also for vector database you can use chromadb as well as shown below. During our experimentation, we found hugging face embeddings and FAISS to be more appropriate for our use case


In [14]:
# google_palm_embeddings = GooglePalmEmbeddings(google_api_key=api_key)

# from langchain.vectorstores import Chroma
# vectordb = Chroma.from_documents(data,
#                            embedding=google_palm_embeddings,
#                            persist_directory='./chromadb')
# vectordb.persist()

### Create RetrievalQA chain along with prompt template 🚀


In [13]:
from langchain.prompts import PromptTemplate

prompt_template = """Given the following context and a question, generate an answer based on this context only.
In the answer try to provide as much text as possible from "response" section in the source document context without making much changes.
If the answer is not found in the context, kindly state "I don't know." Don't try to make up an answer.

CONTEXT: {context}

QUESTION: {question}"""


PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)
chain_type_kwargs = {"prompt": PROMPT}


from langchain.chains import RetrievalQA

chain = RetrievalQA.from_chain_type(llm=llm,
                            chain_type="stuff",
                            retriever=retriever,
                            input_key="query",
                            return_source_documents=True,
                            chain_type_kwargs=chain_type_kwargs)


### We are all set 👍🏼 Let's ask some questions now


In [14]:
chain('Do you provide job assistance and also do you provide job gurantee?')

/Users/meet/anaconda3/envs/omdena/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


{'query': 'Do you provide job assistance and also do you provide job gurantee?',
 'result': 'Yes, We help you with resume and interview preparation along with that we help you in building online credibility, and based on requirements we refer candidates to potential recruiters. The courses included in this bootcamp are done by 9000+ learners and many of them have secured a job which gives us ample confidence that you will be able to get a job. However, we want to be honest and do not want to make any impractical promises! Our guarantee is to prepare you for the job market by teaching the most relevant skills, knowledge & timeless principles good enough to fetch the job.',
 'source_documents': [Document(page_content='prompt: Do you provide any job assistance?\nresponse: Yes, We help you with resume and interview preparation along with that we help you in building online credibility, and based on requirements we refer candidates to potential recruiters.', metadata={'source': 'Do you prov

**As you can see above, the answer of question comes from two different FAQs within our csv file and it is able to pull those questions and merge them nicely**


In [15]:
chain("Do you guys provide internship and also do you offer EMI payments?")

{'query': 'Do you guys provide internship and also do you offer EMI payments?',
 'result': "Yes, we provide virtual internship and we don't offer EMI payments.",
 'source_documents': [Document(page_content='prompt: Do you provide any virtual internship?\nresponse: Yes', metadata={'source': 'Do you provide any virtual internship?', 'row': 14}),
  Document(page_content='prompt: Do we have an EMI option?\nresponse: No', metadata={'source': 'Do we have an EMI option?', 'row': 13}),
  Document(page_content='prompt: Do you provide any job assistance?\nresponse: Yes, We help you with resume and interview preparation along with that we help you in building online credibility, and based on requirements we refer candidates to potential recruiters.', metadata={'source': 'Do you provide any job assistance?', 'row': 11}),
  Document(page_content='prompt: How can I contact the instructors for any doubts/support?\nresponse: We have created every lecture with a motive to explain everything in an easy-

In [16]:
chain("do you have javascript course?")

{'query': 'do you have javascript course?',
 'result': "I don't know.",
 'source_documents': [Document(page_content='prompt: I have never done programming and belong to a non-technical background. Can I take this course?\nresponse: Yes, this is the perfect course for anyone who has never done coding and wants to build a career in the IT/Data Analytics industry or just wants to perform better in their current job or business using data.', metadata={'source': 'I have never done programming and belong to a non-technical background. Can I take this course?', 'row': 24}),
  Document(page_content='prompt: I have never done programming in my life. Can I take this bootcamp?\nresponse: Yes, this is the perfect bootcamp for anyone who has never done coding and wants to build a career in the IT/Data Analytics industry or just wants to perform better in your current job or business using data.', metadata={'source': 'I have never done programming in my life. Can I take this bootcamp?', 'row': 0}),


In [17]:
chain("Do you have plans to launch blockchain course in future?")

{'query': 'Do you have plans to launch blockchain course in future?',
 'result': 'response: We are currently working on launching courses on Data Science, Machine Learning, and Artificial Intelligence. We will launch courses on other domains as well in the future.',
 'source_documents': [Document(page_content='prompt: Will the course be upgraded when there are new features in Power BI?\nresponse: Yes, the course will be upgraded periodically based on the new features in Power BI, and learners who have already bought this course will have free access to the upgrades.', metadata={'source': 'Will the course be upgraded when there are new features in Power BI?', 'row': 27}),
  Document(page_content='prompt: What business concepts and domains are covered in this course?\nresponse: We have covered the core functions such as Sales, Marketing, Finance, and Supply Chain with their fundamentals related to this course. The domain you will learn in this course is consumer goods which is projected 

In [18]:
chain("should I learn power bi or tableau?")

{'query': 'should I learn power bi or tableau?',
 'result': 'This is a contextual question. If you are talking about a pure visualization tool Tableau is slightly better. Data connectors, modeling and transformation features are available in both. However, factually speaking Power BI is cheaper and offers tighter integration with the Microsoft environment. Since most companies use excel & Microsoft tools they start with Power BI or move towards Power BI for seamless integration with other Microsoft tools (called as Power platform). This makes the job openings grow at a much higher rate on Power BI and Power Platform. Also, Power BI has been leading the Gartnerís magic quadrant in BI for the last few years as the industry leader.',
 'source_documents': [Document(page_content='prompt: Power BI or Tableau which one is better?\nresponse: This is a contextual question. If you are talking about a pure visualization tool Tableau is slightly better. Data connectors, modeling and transformation

In [19]:
chain("I've a MAC computer. Can I use powerbi on it?")

{'query': "I've a MAC computer. Can I use powerbi on it?",
 'result': 'response: Hi\n\nPower BI desktop works only in Windows OS. Please look into the system requirements section on this page. However, you can use a virtual machine to install and work with Power BI in other Operating systems.',
 'source_documents': [Document(page_content='prompt: How can I use PowerBI on my Mac system?\nresponse: Hi\n\nYou can use VirtualBox to create a virtual machine and install Windows on it. This will allow you to run Power BI and Excel on your Mac.\n\nIf you\'re not familiar with setting up a virtual machine, there are many resources available on YouTube that can guide you through the process. Simply search for "installing virtual machines" and you\'ll find plenty of helpful videos.\n\nBest of luck with your studies!', metadata={'source': 'How can I use PowerBI on my Mac system?', 'row': 44}),
  Document(page_content='prompt: Does Power BI work in Mac OS/Ubuntu?\nresponse: Power BI desktop works o

In [20]:
chain("I don't see power pivot. how can I enable it?")

{'query': "I don't see power pivot. how can I enable it?",
 'result': 'Follow this thread for instructions - https://support.microsoft.com/en-us/office/start-the-power-pivot-add-in-for-excel-a891a66d-36e3-43fc-81e8-fc4798f39ea8\nIf it doesn\'t show in the ribbon then go to "insert" tab. You will be able to see pivot table option there.',
 'source_documents': [Document(page_content='prompt: How to install power pivot if its not available in system?\nresponse: Follow this thread for instructions - https://support.microsoft.com/en-us/office/start-the-power-pivot-add-in-for-excel-a891a66d-36e3-43fc-81e8-fc4798f39ea8\nIf it doesn\'t show in the ribbon then go to "insert" tab. You will be able to see pivot table option there.', metadata={'source': 'How to install power pivot if its not available in system?', 'row': 38}),
  Document(page_content='prompt: How do I enable Power Pivot before using it for the first time ?\nresponse: Follow the process in the link : \n\nhttps://drive.google.com/fi

In [21]:
chain("What is the price of your machine learning course?")

{'query': 'What is the price of your machine learning course?',
 'result': 'The course is priced at 2999 INR.',
 'source_documents': [Document(page_content='prompt: Iím not sure if this course is good enough for me to invest some money. What can I do?\nresponse: Donít worry. Many videos in this course are free so watch them to get an idea of the quality of teaching. Dhaval Patel (the course instructor) runs a popular data science YouTube channel called Codebasics. On that, you can watch his videos and read comments to get an idea of his teaching style', metadata={'source': 'Iím not sure if this course is good enough for me to invest some money. What can I do?', 'row': 20}),
  Document(page_content='prompt: Will the course be upgraded when there are new features in Power BI?\nresponse: Yes, the course will be upgraded periodically based on the new features in Power BI, and learners who have already bought this course will have free access to the upgrades.', metadata={'source': 'Will the